# About
This was my original and only approach that I submitted to the competition. Initially it got me under #100 (~ #75 rank). I might improve it since we still have 2+ days left at it. I don't know my current rank, it has it's ups and downs, so only private test set analysis will tell. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

import os
import cv2

Append the complete file path to the file name in "filename" column so we can make a submission file.

In [ ]:
DIR = "../input/friendshipgoals/data/test"
test = pd.read_csv("../input/friendshipgoals/data/Test.csv")
test['Category'] = -1
test['Filename'] = test['Filename'].apply(lambda x: os.path.join(DIR, x))
test.head()

Make image datagenerator for training files. No validation for now, just training.

In [ ]:
train_data = ImageDataGenerator()
train = train_data.flow_from_directory(
    directory="../input/friendshipgoals/data/train/",
    target_size=(128, 128),
)

Simple model, nothing fancy here.

In [ ]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(
    train,
    epochs=35
)

In my initial runs, I achieved complete saturation at about 35 epochs. It may vary for you depending on weights randomness, etc.

Save the model

In [ ]:
model.save("friendship_goal_model.h5")

Get the predictions and save them into submission file.

In [ ]:
idx2cat = {0:'Adults', 1:'Teenagers', 2:'Toddler'}
cat = []
for img_name in test['Filename']:
    img = cv2.imread(img_name)
    img = cv2.resize(img, (128, 128))
    cls = model.predict_classes(np.array([img])).tolist()[0]
    cat.append(idx2cat[cls]) 

Save submission to a csv file.

In [ ]:
new_test = pd.read_csv("../input/friendshipgoals/data/Test.csv")
submission = pd.DataFrame({'Filename':new_test['Filename'], 'Category':cat})
submission.to_csv("friendship_goals_sub.csv", index=False)